In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"jawaharrb","key":"28009ebdb4427ca42e824e8be751aca3"}'}

In [ ]:
!ls -lha kaggle.json

-rw-r--r-- 1 root root 65 Aug 27 08:15 kaggle.json


In [ ]:
!pip install -q kaggle


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

 78% 20.0M/25.7M [00:00<00:00, 49.1MB/s]
100% 25.7M/25.7M [00:00<00:00, 65.0MB/s]


In [ ]:
!unzip imdb-dataset-of-50k-movie-reviews.zip

Archive:  imdb-dataset-of-50k-movie-reviews.zip
  inflating: IMDB Dataset.csv        


In [ ]:
imdbreviews=pd.read_csv('IMDB Dataset.csv')

In [ ]:
imdbreviews.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
imdbreviews.sentiment=imdbreviews.sentiment.apply(lambda x: 1 if x=='positive' else 0)
imdbreviews.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [ ]:
import re
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
corpus = []
for i in range(0, len(imdbreviews)):
    review = re.sub('[^a-zA-Z]', ' ', imdbreviews['review'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
voc_size=10000

In [ ]:
onehotrep=[one_hot(words,voc_size) for words in corpus]

In [ ]:
num_tokens=[len(tokens) for tokens in onehotrep]
num_tokens=np.array(num_tokens)

In [ ]:
np.mean(num_tokens)

122.19208

In [ ]:
np.max(num_tokens)

1442

In [ ]:
max_tokens= np.mean(num_tokens) + 2*np.std(num_tokens)
max_tokens=int(max_tokens)
max_tokens

307

In [ ]:
np.sum(num_tokens<max_tokens)/len(num_tokens)

0.94506

In [ ]:
pad='pre'
x= pad_sequences(onehotrep, maxlen=max_tokens, padding=pad, truncating=pad)

In [ ]:
x=np.array(x)
y=np.array(imdbreviews.sentiment)

In [ ]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest= train_test_split(x,y,test_size=0.2,random_state=0)

In [ ]:
embeddingsize=100


In [ ]:
model=Sequential()
model.add(Embedding(voc_size,embeddingsize,input_length=max_tokens))
model.add(LSTM(32,return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(32))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))



In [ ]:
opt=Adam(lr=1e-3)

In [ ]:
model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 307, 100)          1000000   
_________________________________________________________________
lstm_14 (LSTM)               (None, 307, 32)           17024     
_________________________________________________________________
dropout_12 (Dropout)         (None, 307, 32)           0         
_________________________________________________________________
lstm_15 (LSTM)               (None, 32)                8320      
_________________________________________________________________
dropout_13 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 33        
Total params: 1,025,377
Trainable params: 1,025,377
Non-trainable params: 0
____________________________________________

In [ ]:
model.fit(xtrain,ytrain,validation_data=(xtest,ytest),epochs=3,batch_size=64)

Epoch 1/3
625/625 [==============================] - 93s 148ms/step - loss: 0.3810 - accuracy: 0.8316 - val_loss: 0.3067 - val_accuracy: 0.8736
Epoch 2/3
625/625 [==============================] - 92s 147ms/step - loss: 0.2538 - accuracy: 0.9004 - val_loss: 0.3215 - val_accuracy: 0.8723
Epoch 3/3
625/625 [==============================] - 92s 148ms/step - loss: 0.2035 - accuracy: 0.9237 - val_loss: 0.3488 - val_accuracy: 0.8652


In [ ]:
ypred=model.predict_classes(xtest)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [77]:
from sklearn.metrics import accuracy_score
accuracy_score(ytest,ypred)

0.8652